In [1]:
import warnings
warnings.filterwarnings(action='ignore') 

import csv
import folium
import matplotlib.pyplot as plt
plt.rc('font',family='D2CodingLigature Nerd Font')

## 인구수 총합을 내림차순 정렬하여 위 10개만 지도에 표시하기

#### 1. 파일 읽어오기

In [2]:
# 인구 데이터 읽어와서 리스트로 저장하기
f1 = open('../../data/LOCAL_PEOPLE_DONG_202510.csv', encoding='utf8')
data = csv.reader(f1)
next(data)
data = list(data)

# 행정동코드 데이터 불러와서 리스트로 저장하기
f2 = open('../../data/dong_code.csv')
code_data = csv.reader(f2)
next(code_data)
next(code_data)
code_data = list(code_data)

# 위치(위도,경도) 데이터 불러와서 리스트로 저장하기
f3 = open('../../data/dong_location.csv')
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

In [3]:
# 인구데이터의 데이터타입 변경하기 
for row in data: 
    for i in range(1, 32): 
        if i <= 2: 
            row[i] = int(row[i])
        else: 
            row[i] = float(row[i])

# 행정동코드 데이터의 데이터 타입 변환하기
for row in code_data:
    row[1] = int(row[1])        # 8자리 행자부행정동코드

# 위치 정보 데이터의 데이터 타입 변환하기
for row in location_data:       # row[-2]:경도, row[-1]:위도
    row[-2], row[-1] = float(row[-2]), float(row[-1])

#### 2. 행정동별 인구수를 저장하는 빈 딕셔너리를 만들고 행정동별 총생활인구를 딕셔너리에 저장하기

In [6]:
dong_population = {}

for row in data: 
    dong_code, p = row[2], row[3]
    if dong_code not in dong_population.keys(): 
        dong_population[dong_code] = p
    else:
        dong_population[dong_code] += p
        
print(len(dong_population))     # 완성된 딕셔너리의 요소 갯수
print(dong_population)

424
{11110515: 12468503.275399998, 11110530: 17816941.92039999, 11110540: 4840370.374100002, 11110550: 8237357.6709, 11110560: 13262475.566800002, 11110570: 4634208.875299993, 11110580: 5609224.401500001, 11110600: 4358449.1689, 11110615: 44998987.88349998, 11110630: 10051354.21930001, 11110640: 20511858.43820002, 11110650: 23021213.278999984, 11110670: 5837624.500399997, 11110680: 5395823.026600003, 11110690: 2585917.3392000017, 11110700: 5990893.347300004, 11110710: 8610504.241200002, 11140520: 18613371.6715, 11140540: 18465866.620699983, 11140550: 27553458.442700006, 11140570: 11096999.944700012, 11140580: 8067545.439399994, 11140590: 17893179.328699987, 11140605: 11408708.03739999, 11140615: 14507729.710400004, 11140625: 10640728.286699988, 11140635: 9902388.53320001, 11140645: 6785668.681300001, 11140650: 4717489.750900003, 11140665: 9018450.907000005, 11140670: 8687146.454599993, 11140680: 6194493.599699997, 11170510: 9110859.540599985, 11170520: 8999974.621599998, 11170530: 1205

#### 3. 인구수를 기준으로 내림차순 정렬하고, 앞에서 10개 항목만 별도로 저장하기

In [ ]:
import operator     # operator.itemgetter

top10 = sorted(dong_population.items(), key=operator.itemgetter(1), reverse = True)[:10]
print(top10)
print("")

top10_code8 = [t[0] for t in top10]    # 상위 10개의 키만
print(top10_code8)

[(11680640, 69110171.72540002), (11560540, 67715789.74729997), (11440660, 51265783.43090001), (11650530, 45776786.07059998), (11110615, 44998987.88349998), (11740685, 43887517.473199956), (11410585, 41127153.3296), (11545510, 40875672.275199994), (11380690, 40827063.7269), (11410555, 40141642.33100002)]

[11680640, 11560540, 11440660, 11650530, 11110615, 11740685, 11410585, 11545510, 11380690, 11410555]


In [9]:
# 각 요소가 0으로 초기화한 크기가 10인크기가 10인  리스트

top10_name = [0 for i in range(10)]
top10_lat  = [0 for i in range(10)]
top10_long = [0 for i in range(10)]

#### 4. 8자리 행정동코드에 해당하는 행정동명 찾기 & 위도와 경도를 찾아 리스트에 저장

In [ ]:
for i in range(len(top10_code8)): 
    for row in code_data: 
        if row[1] == top10_code8[i]:  
            # 행정동명 리스트에 저장하기 
            top10_name[i] = row[-1]
    
    for row in location_data:    ## 위도와 경도를 찾아 리스트에 저장
        if row[3] == top10_name[i]:
            top10_lat[i], top10_long[i] = row[-1], row[-2] 

print(top10_name)   # 이름
print(top10_lat)    # 위도
print(top10_long)   # 경도

['역삼1동', '여의도동', '서교동', '서초3동', '종로1.2.3.4가동', '길동', '신촌동', '가산동', '진관동', '북아현동']
[37.4999211, 37.5286662, 37.5550337, 37.4856591, 37.5777701, 37.5398758, 37.5659638, 37.4756389, 37.64193, 37.5593178]
[127.0367584, 126.9290731, 126.9228533, 127.0097616, 126.9892737, 127.1473638, 126.9384315, 126.8850721, 126.9387277, 126.9551002]


#### 5. 완성된 리스트 내용으로 지도에 표시하기

In [ ]:
import folium

top10_name_r =[0   for i in range(10)]

for i in range(len(top10_name)) : 
    top10_name_r[i] = str(i+1)+'. '+top10_name[i]

    
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10): 
    folium.Marker(
        [top10_lat[i], top10_long[i]], tooltip=top10_name_r[i], icon=folium.Icon(color='red', icon='star')
        ).add_to(map_top10)

map_top10